In [ ]:
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold
import scipy.stats as stat
import pylab
import xgboost as xgb
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
X = pd.read_csv("Train_XBoB1.csv",usecols=['temp','sal','mld','chla','dic','no3'])
y = pd.read_csv("Train_yBoB1.csv",usecols=['misfit'])
X_test = pd.read_csv("Test_XBoB1.csv",usecols=['temp','sal','mld','chla','dic','no3'])
y_test = pd.read_csv("Test_yBoB1.csv",usecols=['misfit'])

X['chla'] =np.log(X['chla'])
X['no3'] =np.log(X['no3'])
X['mld'] =np.log(X['mld'])

X_test['mld'] =np.log(X_test['mld'])
X_test['no3'] =np.log(X_test['no3'])
X_test['chla'] =np.log(X_test['chla'])


In [ ]:
# sc = StandardScaler()
# X = sc.fit_transform(X)
# X_test = sc.transform(X_test)

In [ ]:
kfold = KFold(n_splits=10)

In [ ]:
#Used for prediction and written......................
params={'lambda': 0.8040219538686119,
 'alpha': 0.31013667167623227,
 'subsample': 0.8340547614201859,
 'colsample_bytree': 0.9780461930190425,
 'max_depth': 25,
 'min_child_weight': 28,
 'learning_rate': 0.09942607393074405,
 'gamma': 0.03187690049095798,
 'n_estimators': 500,'random_state':0}

In [ ]:
for train_ix, val_ix in kfold.split(X):
    X_train, X_val = X.iloc[train_ix], X.iloc[val_ix]
    y_train, y_val = y.iloc[train_ix], y.iloc[val_ix]
    # model = xgb.XGBRegressor()
    model = xgb.XGBRegressor(**params)
    model.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_val,y_val)])

In [ ]:
y_pred = model.predict(X_test)
r21 = r2_score(y_test,y_pred)
print(r21)
rmse = mean_squared_error(y_test,y_pred,squared=False)
print(rmse)

In [ ]:
y_pred_train = model.predict(X_train)
r2_tr = r2_score(y_train,y_pred_train)
print(r2_tr)
rmse_tr = mean_squared_error(y_train,y_pred_train,squared=False)
print(rmse_tr)

mae_tr = mean_absolute_error(y_train,y_pred_train)
print(mae_tr)

In [ ]:
results = model.evals_result()
fig, ax = plt.subplots()
ax.plot( results['validation_0']['rmse'], label='Train')
ax.plot( results['validation_1']['rmse'], label='Test')
ax.legend()
plt.ylabel('rmse')
plt.title('XGBoost')

In [ ]:
fig,ax=plt.subplots(figsize=(14,3))
ax=ax.margins(x=0)
plt.plot(y_test[0:50])
plt.plot(y_pred[0:50])

In [ ]:
from joblib import dump
# dump(model,"xgb_SD_BoB.joblib_correctednew.dat")